<a href="https://colab.research.google.com/github/etuckerman/LLaMA-3.1-QA-Finetune/blob/main/unsloth_llama_3_1_70b_instruct.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install huggingface_hub

Defaulting to user installation because normal site-packages is not writeable


In [ ]:
from huggingface_hub import login
login(token="HIDDEN_FOR_OBVIOUS_REASONS")

/home/ubuntu/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /home/ubuntu/.cache/huggingface/token
Login successful


In [ ]:
!pip install -r requirements.txt

Defaulting to user installation because normal site-packages is not writeable
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 66.5 MB/s eta 0:00:0000:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
ERROR: Ignored the following versions that require a different python version: 0.0.1a1 Requires-Python >=3.6, <3.10; 0.0.1a2 Requires-Python >=3.6, <3.10; 0.1.0 Requires-Python >=3.6, <3.10; 0.1.1 Requires-Python >=3.6, <3.10; 0.2.0 Requires-Python >=3.6, <3.10
ERROR: Could not find a version that satisfies the requirement en-core-web-sm==3.7.1 (from versions: none)
ERROR: No matching distribution found for en-core-web-sm==3.7.1


In [ ]:
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
from torch import __version__; from packaging.version import Version as V
xformers = "xformers==0.0.27" if V(__version__) < V("2.4.0") else "xformers"
!pip install --no-deps {xformers} trl peft accelerate bitsandbytes triton


Defaulting to user installation because normal site-packages is not writeable
  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-plehlcgr/unsloth_26f221892c2e4b59ae9419b6cdbd7dad
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-plehlcgr/unsloth_26f221892c2e4b59ae9419b6cdbd7dad
  Resolved https://github.com/unslothai/unsloth.git to commit 6c534341bb229b136f9504443f0161645d2070c5
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Defaulting to user installation because normal site-packages is not writeable


In [ ]:
import pandas as pd
from datasets import Dataset, DatasetDict
from transformers import LlamaTokenizer, LlamaForCausalLM, Trainer, TrainingArguments
from peft import get_peft_model, LoraConfig, TaskType
import torch

# Load the dataset
df = pd.read_csv('qa_examples.csv')

# Convert to Dataset format
data = Dataset.from_pandas(df)

# Split the dataset
train_size = 12
test_size = 2
val_size = 2

train_data = data.select(range(train_size))
test_data = data.select(range(train_size, train_size + test_size))
val_data = data.select(range(train_size + test_size, train_size + test_size + val_size))

dataset = DatasetDict({
    'train': train_data,
    'test': test_data,
    'validation': val_data
})


In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/Meta-Llama-3.1-8B-bnb-4bit",      # Llama-3.1 15 trillion tokens model 2x faster!
    "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    "unsloth/Meta-Llama-3.1-70B-bnb-4bit",
    "unsloth/Meta-Llama-3.1-405B-bnb-4bit",    # We also uploaded 4bit for 405b!
    "unsloth/Meta-Llama-3.1-70B-Instruct-bnb-4bit",
    "unsloth/Mistral-Nemo-Base-2407-bnb-4bit", # New Mistral 12b 2x faster!
    "unsloth/Mistral-Nemo-Instruct-2407-bnb-4bit",
    "unsloth/mistral-7b-v0.3-bnb-4bit",        # Mistral v3 2x faster!
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/Phi-3.5-mini-instruct",           # Phi-3.5 2x faster!
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/gemma-2-9b-bnb-4bit",
    "unsloth/gemma-2-27b-bnb-4bit",            # Gemma 2x faster!
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-70B-Instruct-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
==((====))==  Unsloth 2024.8: Fast Llama patching. Transformers = 4.44.2.
   \\   /|    GPU: NVIDIA A100 80GB PCIe. Max memory: 79.151 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.4.0+cu121. CUDA = 8.0. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.27.post2. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


Loading checkpoint shards: 100%|██████████| 6/6 [00:07<00:00,  1.17s/it]


In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2024.8 patched 80 layers with 80 QKV layers, 80 O layers and 80 MLP layers.


In [ ]:
alpaca_prompt = """Below is a question. Write the correct answer.

### Question:
{}

### Answer:
{}"""

EOS_TOKEN = tokenizer.eos_token  # Must add EOS_TOKEN

def formatting_prompts_func(examples):
    questions = examples["Question"]
    answers = examples["Answer"]
    texts = []
    for question, answer in zip(questions, answers):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = alpaca_prompt.format(question, answer) + EOS_TOKEN
        texts.append(text)
    return {"text": texts}

from datasets import load_dataset

# Load your dataset
dataset = load_dataset('csv', data_files={'train': 'qa_examples.csv'}, split='train')

# Apply the formatting function
dataset = dataset.map(formatting_prompts_func, batched=True)


Generating train split: 16 examples [00:00, 5667.98 examples/s]
Map: 100%|██████████| 16/16 [00:00<00:00, 5467.56 examples/s]


In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        # num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

Map (num_proc=2): 100%|██████████| 16/16 [00:00<00:00, 23.60 examples/s]
max_steps is given, it will override any value given in num_train_epochs


In [ ]:
#@title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = NVIDIA A100 80GB PCIe. Max memory = 79.151 GB.
38.859 GB of memory reserved.


In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 16 | Num Epochs = 30
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 60
 "-____-"     Number of trainable parameters = 207,093,760


Step,Training Loss
1,2.586600
2,2.506600
3,2.499600
4,2.407300
5,2.102200
6,1.590100
7,1.227700
8,0.933100
9,0.661400
10,0.467600


In [ ]:
#@title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory         /max_memory*100, 3)
lora_percentage = round(used_memory_for_lora/max_memory*100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

360.0789 seconds used for training.
6.0 minutes used for training.
Peak reserved memory = 40.941 GB.
Peak reserved memory for training = 2.082 GB.
Peak reserved memory % of max memory = 51.725 %.
Peak reserved memory for training % of max memory = 2.63 %.


In [ ]:
# Set up for inference
FastLanguageModel.for_inference(model)  # Enable native 2x faster inference

# Define the question you want to test
question = "What is the fuel capacity of a Jayco Eagle RV?"  # Replace with any question from your dataset

# Format the input for the model
inputs = tokenizer(
    [
        alpaca_prompt.format(
            question,  # Replace with the question you want to answer
            "",  # No additional context needed
        )
    ],
    return_tensors="pt"
).to("cuda")

# Generate the answer
outputs = model.generate(
    **inputs,
    max_new_tokens=64,  # Adjust as needed for longer answers
    use_cache=True
)

# Decode and print the generated answer
generated_text = tokenizer.batch_decode(outputs, skip_special_tokens=True)
print(generated_text[0])


Below is a question. Write the correct answer.

### Question:
What is the fuel capacity of a Jayco Eagle RV?

### Answer:
The Jayco Eagle has a fuel capacity of 75 gallons.


In [ ]:
# Set up for inference with native 2x faster inference
FastLanguageModel.for_inference(model)

# Define the question you want to test
question = "Does the Jayco Eagle have a queen size bed?"  # Replace with questions

# Format the input for the model
inputs = tokenizer(
    [
        alpaca_prompt.format(
            question,  # Replace with the question you want to answer
            "",  # No additional context needed
        )
    ],
    return_tensors="pt"
).to("cuda")

# Set up the TextStreamer for continuous output
from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)

# Generate the answer with streaming
_ = model.generate(
    **inputs,
    streamer=text_streamer,
    max_new_tokens=128  # Adjust based on expected length of answers
)


<|begin_of_text|>Below is a question. Write the correct answer.

### Question:
how many queen sizes does the jayco eagle have

### Answer:
The Jayco Eagle has 1 queen size bed.<|eot_id|>


In [ ]:
# Set up for inference with native 2x faster inference
FastLanguageModel.for_inference(model)

# List of questions for inference
questions = [
    "Does the Jayco Eagle have a queen size bed?",
    "What is the MSRP of the Jayco Eagle?",
    "How many people can sleep in the Jayco Eagle?",
    "Does the Jayco Eagle have a shower?",
    "Does the Keystone Cougar have a queen size bed?",
    "What is the MSRP of the Keystone Cougar?",
    "How many people can sleep in the Keystone Cougar?",
    "Does the Keystone Cougar have a shower?",
    "Does the Coachmen Clipper Sport have a queen size bed?",
    "What is the MSRP of the Coachmen Clipper Sport?",
    "How many people can sleep in the Coachmen Clipper Sport?",
    "Does the Coachmen Clipper Sport have a shower?",
    "Does the Fleetwood Fiesta have a queen size bed?",
    "What is the MSRP of the Fleetwood Fiesta?",
    "How many people can sleep in the Fleetwood Fiesta?",
    "Does the Fleetwood Fiesta have a shower?"
]

# Loop through each question
for question in questions:
    print(f"Question: {question}")  # Optional: Display the current question

    # Format the input for the model
    inputs = tokenizer(
        [
            alpaca_prompt.format(
                question,  # Replace with the question you want to answer
                "",  # No additional context needed
            )
        ],
        return_tensors="pt"
    ).to("cuda")

    # Set up the TextStreamer for continuous output
    from transformers import TextStreamer
    text_streamer = TextStreamer(tokenizer)

    # Generate the answer with streaming
    _ = model.generate(
        **inputs,
        streamer=text_streamer,
        max_new_tokens=128  # Adjust based on expected length of answers
    )

    print("\n")  # Add a newline after each answer for better readability


Question: Does the Jayco Eagle have a queen size bed?
<|begin_of_text|>Below is a question. Write the correct answer.

### Question:
Does the Jayco Eagle have a queen size bed?

### Answer:
Yes, the Jayco Eagle has 1 queen size bed.<|eot_id|>


Question: What is the MSRP of the Jayco Eagle?
<|begin_of_text|>Below is a question. Write the correct answer.

### Question:
What is the MSRP of the Jayco Eagle?

### Answer:
The MSRP of the Jayco Eagle is $32,186.<|eot_id|>


Question: How many people can sleep in the Jayco Eagle?
<|begin_of_text|>Below is a question. Write the correct answer.

### Question:
How many people can sleep in the Jayco Eagle?

### Answer:
The Jayco Eagle can sleep up to 9 people.<|eot_id|>


Question: Does the Jayco Eagle have a shower?
<|begin_of_text|>Below is a question. Write the correct answer.

### Question:
Does the Jayco Eagle have a shower?

### Answer:
Yes, the Jayco Eagle comes with a standard shower.<|eot_id|>


Question: Does the Keystone Cougar have a 

In [ ]:
# Save model and tokenizer locally
model.save_pretrained("lora_model")  # Save the model to the "lora_model" directory
tokenizer.save_pretrained("lora_model")  # Save the tokenizer to the "lora_model" directory


('lora_model/tokenizer_config.json',
 'lora_model/special_tokens_map.json',
 'lora_model/tokenizer.json')

In [ ]:


# Push the model to Hugging Face Hub
#model.push_to_hub("username/lora_model")

# Push the tokenizer to Hugging Face Hub
#tokenizer.push_to_hub("username/lora_model")

In [ ]:
#Now if you want to load the LoRA adapters we just saved for inference, set `False` to `True`:

In [ ]:
if False:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "lora_model", # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
    )
    FastLanguageModel.for_inference(model) # Enable native 2x faster inference

alpaca_prompt = """Below is a question. Write the correct answer.

### Question:
{}

### Answer:
{}"""

inputs = tokenizer(
[
    alpaca_prompt.format(
        "Does the Jayco Eagle have a queen size bed?", # instruction
        "", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

<|begin_of_text|>Below is a question. Write the correct answer.

### Question:
Does the Jayco Eagle have a queen size bed?

### Answer:
Yes, the Jayco Eagle has 1 queen size bed.<|eot_id|>


In [ ]:
from huggingface_hub import login
login(token="HIDDEN_FOR_OBVIOUS_REASONS")

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /home/ubuntu/.cache/huggingface/token
Login successful


In [ ]:
# Upload the model
model.push_to_hub("etuckerman/Llama-3.1-70B-Instruct-Finetuned")
tokenizer.push_to_hub("etuckerman/Llama-3.1-70B-Instruct-Finetuned")

print("Model and tokenizer have been uploaded to Hugging Face!")

adapter_model.safetensors: 100%|██████████| 829M/829M [00:21<00:00, 38.3MB/s]   


Saved model to https://huggingface.co/etuckerman/Llama-3.1-70B-Instruct-Finetuned
Model and tokenizer have been uploaded to Hugging Face!
